### Prediction Model

Data has been already cleaned in SQL.

A database with a timestamp every 5 min was created with the station and weather data.
The day-time savings time change was adjusted.
A column for the day was added.
The status for the night was corrected to closed.
The CSV clean_db.csv was created from that. This database will be used as test and validation set for this prediction model.
Column with just time.

For the prediction model there will be a random forest regression done for each station.

In [1]:
# Import the required packages
# Import package pandas for data analysis
import pandas as pd

# Import package numpy for numeric computing
import numpy as np

# Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt

# Imports for random forest regression
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor

# For showing plots directly in the notebook run the command below
%matplotlib inline

# Connect DB
import sys
sys.path.append('../web/')
from dbConnection import connect_db, get_clean_db


# Pickle
import pickle

In [ ]:
# Reading from a csv file, into a data frame
df = pd.read_csv("/Users/florian/Documents/GitHub/clean_db.csv", keep_default_na=True, dtype={16: str}, delimiter=',', skipinitialspace=True, encoding='Windows-1252') #sep=',\s+',

In [ ]:
#sql_query = get_clean_db()
#df = pd.DataFrame(sql_query, columns = ['timestamp', 'station_id', 'available_bikes', 'available_bike_stands', 'status', 'temperature', 'pressure', 'humidity', 'clouds', 'wind_speed_beaufort', 'wind_direction', 'precipitation_value', 'precipitation_min', 'precipitation_max', 'precipitation_probability', 'wind_speed_mps', 'weather_type', 'icon_number', 'temperature_feels_like', 'day_flag', 'time', 'day'])

In [ ]:
print("Rows: " + str(df.shape[0]))
print("Columns: " + str(df.shape[1]))
print(df.columns)
df.dtypes

### Time Changes

In [ ]:
# Type changes & drops
df.drop(['weather_type', 'icon_number'], axis=1, inplace=True)

In [ ]:
df["time"] = pd.to_datetime(df["time"], format="%H:%M:%S")
df["time"] = df["time"].dt.strftime("%H:%M:%S")

In [ ]:
def to_category(column):
    list_unique = []
    for i in sorted(df[column].unique()):
        list_unique.append(i)

    list_range = [i for i in range(len(list_unique))]
    df[column] = df[column].astype('category')
    df[column].cat.add_categories(list_range, inplace=True)

    df[column] = df[column].replace(list_unique, list_range)
    df[column] = df[column].astype('int64')

In [ ]:
to_category("status")
to_category("time")
to_category("day")

In [ ]:
print(df["status"].ravel())
print(df["time"].ravel())
print(df["day"].ravel())
print(df.dtypes)

In [ ]:
# Ratio of availability

df['availabilty_ratio'] = df['available_bikes'] / (df['available_bikes'] + df['available_bike_stands'])

In [ ]:
df.loc[df['availabilty_ratio'].isnull()]

-> Seems to be an error or a time where station is closed if bike and stands is 0. Those rows will be dropped

In [ ]:
df = df.loc[df['availabilty_ratio'].notnull()]

In [ ]:
df['max_stands'] = df['available_bikes'] + df['available_bike_stands']
stations_unique = sorted(df["station_id"].unique())
for station in stations_unique:
    df_station = df.loc[df["station_id"] == station]
    df_clean_station = df_station.loc[df_station["max_stands"] == df_station["max_stands"].median()]
    print(station, "min:", df_station["max_stands"].min(), "mean:", df_station["max_stands"].median(),"max:", df_station["max_stands"].max(),"-", df_station.shape[0] - df_clean_station.shape[0])

In [ ]:
print(df.loc[(df["max_stands"].median()-3 < df["max_stands"]) & (df["max_stands"] < df["max_stands"].median()+3)].shape[0])
print(df.shape[0])

-> Seems like many rows don't add up

In [ ]:
for station in stations_unique:
    df.loc[df["station_id"] == station, 'max_stands'] = round(df.loc[df["station_id"] == station, 'max_stands'].median())

for station in stations_unique:
    df_station = df.loc[df["station_id"] == station]
    df_clean_station = df_station.loc[df_station["max_stands"] == df_station["max_stands"].median()]
    print(station, "min:", df_station["max_stands"].min(), "mean:", df_station["max_stands"].median(),"max:", df_station["max_stands"].max(),"-", df_station.shape[0] - df_clean_station.shape[0])

## Redo: Ratio:

In [ ]:
df['availabilty_ratio'] = df['available_bikes'] / df['max_stands']

In [ ]:
def correlation(df):
    # Correlation matrix using code found on https://stanford.edu/~mwaskom/software/seaborn/examples/many_pairwise_correlations.html
    sns.set(style="white")

    # Calculate correlation of all pairs of continuous features
    corr = df.corr()

    # Generate a mask for the upper triangle
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # Generate a custom colormap - blue and red
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, annot=True, mask=mask, cmap=cmap, vmax=1, vmin=-1,
                square=True, xticklabels=True, yticklabels=True,
                linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)
    plt.yticks(rotation = 0)
    plt.xticks(rotation = 45)

In [ ]:
df_station = df.loc[df["station_id"] == 1]
#correlation(df_station)
corr = df.corr()
corr

In [ ]:
correlation_tables = []
for station in stations_unique:
    df_station = df.loc[df["station_id"] == station]
    correlation_tables.append(df_station.corr())

# Concatenate the correlation tables into a single dataframe
merged_corr_table = pd.concat(correlation_tables)
correlation(merged_corr_table)

# average correlation - in relation to each station

Result:
- precipitation_min / _max / _value seem to represent a similar value with similar correlation. Will be reduced to just the _value.
- wind direction and speed seem to have little influence and will be dropped as well
- all rows with status "CLOSED" will be dropped and then the column will be dropped - not relevant

In [ ]:
df = df.loc[df["status"] == 1]
df.drop(['precipitation_min', 'precipitation_max', 'wind_speed_mps', 'wind_direction', 'wind_speed_beaufort', 'status'], axis=1, inplace=True)

In [ ]:
df.describe().T

In [ ]:
df.isnull().sum()

In [ ]:
# drop temperature_feels_like and day_flag as we don't have them for future values:
df = df.drop(['temperature_feels_like', 'day_flag'],axis=1)

Adding Flag columns for chance:

In [ ]:
df['bikes_flag'] = df['available_bikes'].apply(lambda x: 0 if x == 0 else 1)
df['stands_flag'] = df['available_bike_stands'].apply(lambda x: 0 if x == 0 else 1)
df.describe().T

Historical Data db:

In [ ]:
prediction_clean_df = df.drop(['temperature', 'pressure', 'humidity', 'clouds', 'precipitation_value', 'precipitation_probability'],axis=1)
prediction_clean_df.to_csv("prediction_clean_df.csv", index=False)
prediction_clean_df.columns

In [ ]:
# Write clean_db in CSV
df.drop(['timestamp'], axis=1, inplace=True)
df.to_csv("cleaned_clean_db.csv", index=False)

## Random Forest Experiments

As a first test an overall model:

In [ ]:
X = df.drop(['station_id', 'available_bikes', 'available_bike_stands', 'availabilty_ratio', 'max_stands', 'bikes_flag', 'stands_flag'],axis=1)
y = df['availabilty_ratio']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

In [ ]:
regr = RandomForestRegressor(n_estimators = 100, random_state = 42)

In [ ]:
%%time
regr.fit(X_train, y_train)

In [ ]:
# checking the prediction
y_pred = regr.predict(X_test)

print("Mean absolute error:", metrics.mean_absolute_error(y_test, y_pred))
print("Root Mean Square Error:", metrics.mean_squared_error(y_test, y_pred)**0.5)

-> Not really precise. 0.3 root mean square error for a value between 0 and 1.

In [ ]:
plt.scatter(y_test, y_pred)
plt.ylabel('Predicted DT')
plt.xlabel('Actual DT')

-> Prediction seems to be really inaccurate.

#### Random Search Training
Source for following parameter tuning: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor(random_state = 42)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 5, cv = 2, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
# Best parameters
rf_random.best_params_

#### Evaluate Random Search
To determine if random search yielded a better model, we compare the base model with the best random search model

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    RMSE = metrics.mean_squared_error(test_labels, regr.predict(test_features))**0.5
    errors = abs(predictions - test_labels)
    #mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - (100*RMSE)
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('RMSE: {:0.4f}.'.format(RMSE))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy
base_model = regr
base_accuracy = evaluate(base_model, X_test, y_test)

In [ ]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)

In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

--> not worth the perfomance needed to improve the model.

### Individual Models

In [ ]:
# First test with the first stations and a plot

prediction_models = {}
for station in stations_unique[:5]:
        df_station = df.loc[df["station_id"] == station]
        X = df_station.drop(['station_id', 'available_bikes', 'available_bike_stands', 'availabilty_ratio', 'max_stands', 'bikes_flag', 'stands_flag'],axis=1)
        y = df_station['availabilty_ratio']

        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)
        
        regr = RandomForestRegressor(n_estimators = 100, random_state = 42)
        regr.fit(X_train, y_train)

        prediction_models[station] = regr

        y_pred = regr.predict(X_test)

        print("Mean absolute error:", metrics.mean_absolute_error(y_test, y_pred))
        print("Root Mean Square Error:", metrics.mean_squared_error(y_test, y_pred)**0.5)

        plt.scatter(y_test, y_pred)
        plt.ylabel('Predicted DT')
        plt.xlabel('Actual DT')

Specific models seem to be much better.

In [ ]:
# Test of training models:
def training(base_model, X_train, X_test, y_train, y_test):
        # Number of trees in random forest
        n_estimators = [int(x) for x in np.linspace(start = 10, stop = 1000, num = 10)]
        # Number of features to consider at every split
        max_features = ['auto', 'sqrt']
        # Maximum number of levels in tree
        max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
        max_depth.append(None)
        # Minimum number of samples required to split a node
        min_samples_split = [2, 5, 10]
        # Minimum number of samples required at each leaf node
        min_samples_leaf = [1, 2, 4]
        # Method of selecting samples for training each tree
        bootstrap = [True, False]
        # Create the random grid
        random_grid = {'n_estimators': n_estimators,
                'max_features': max_features,
                'max_depth': max_depth,
                'min_samples_split': min_samples_split,
                'min_samples_leaf': min_samples_leaf,
                'bootstrap': bootstrap}

        # Use the random grid to search for best hyperparameters
        # First create the base model to tune
        rf = RandomForestRegressor(random_state = 42)
        # Random search of parameters, using 3 fold cross validation, 
        # search across 100 different combinations, and use all available cores
        rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=42, n_jobs = -1)
        # Fit the random search model
        rf_random.fit(X_train, y_train)

        base_accuracy = evaluate(base_model, X_test, y_test)
        best_random = rf_random.best_estimator_
        random_accuracy = evaluate(best_random, X_test, y_test)
        print('----------------------------------------------------------')
        print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))
        print('----------------------------------------------------------')


prediction_models = {}
for station in stations_unique[:5]:
        df_station = df.loc[df["station_id"] == station]
        X = df_station.drop(['station_id', 'available_bikes', 'available_bike_stands', 'availabilty_ratio', 'max_stands'],axis=1)
        y = df_station['availabilty_ratio']

        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)
        
        regr = RandomForestRegressor(n_estimators = 100, random_state = 42)
        regr.fit(X_train, y_train)

        prediction_models[station] = regr

        y_pred = regr.predict(X_test)

        print("Mean absolute error:", metrics.mean_absolute_error(y_test, y_pred))
        print("Root Mean Square Error:", metrics.mean_squared_error(y_test, y_pred)**0.5)

        training(regr, X_train, X_test, y_train, y_test)


Result: For this sample the training did not improve the accuracy, so we won't train each model to reduce on performance needs.

In [ ]:
prediction_models = {}
for station in stations_unique:
        df_station = df.loc[df["station_id"] == station]
        max_stands = df.loc[df["station_id"] == station]["max_stands"].max()
        X = df_station.drop(['station_id', 'available_bikes', 'available_bike_stands', 'availabilty_ratio', 'max_stands', 'bikes_flag', 'stands_flag'],axis=1)
        y = df_station['availabilty_ratio']
        y_bike_chance = df_station['bikes_flag']
        y_stands_chance = df_station['stands_flag']

        #X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)
        
        regr = RandomForestRegressor(n_estimators = 100, random_state = 42)
        regr.fit(X, y)

        regr_bike_chance = RandomForestRegressor(n_estimators = 100, random_state = 42)
        regr_bike_chance.fit(X, y_bike_chance)

        regr_stands_chance = RandomForestRegressor(n_estimators = 100, random_state = 42)
        regr_stands_chance.fit(X, y_stands_chance)

        prediction_models[station] = [regr.predict, max_stands, regr_bike_chance.predict, regr_stands_chance.predict]



In [ ]:
# Test prediction
# ['temperature', 'pressure', 'humidity', 'clouds', 'precipitation_value', 'precipitation_probability', 'time', 'day']
test_values = np.array([[6, 1026.5, 60.4, 79.9, 0, 0, 150, 4]])
pred = prediction_models[1][0](test_values)
pred_chance = prediction_models[1][2](test_values)
pred_stand_chance = prediction_models[1][3](test_values)
print(pred)
print(float(pred_chance))
print(pred_stand_chance)
print(pred * prediction_models[1][1])


In [ ]:
def get_available_bike_prediction(station_id, date, temperature, pressure, humidity, clouds, precipitation_value, precipitation_probability):
   """ date as string: '2023-05-02 23:59:59' """
   # time conversion:
   # hours * 12 + minutes//5 * 1
   time = int(date[11:13]) * 12 + int(date[14:16])//5
   # day conversion:
   d = pd.Timestamp(date[:10])
   day = d.dayofweek

   input_value = np.array([[temperature, pressure, humidity, clouds, precipitation_value, precipitation_probability, time, day]])
   pred = int(prediction_models[station_id][0](input_value))
   pred_bikes = pred * prediction_models[station_id][1]
   pred_stations = prediction_models[station_id][1]-pred_bikes
   return pred_bikes, pred_stations

In [ ]:
bike, stands = get_available_bike_prediction(1, '2023-05-02 23:59:59', 6, 1026.5, 60.4, 79.9, 0, 0)
print(bike, stands)

0 31


/Users/florian/opt/anaconda3/envs/comp30830/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [44]:
# Export function:
with open('predictionModels.pkl', 'wb') as file:
    pickle.dump(prediction_models, file)

In [29]:
# Code for pickle file per station:
prediction_models = {}
for station in stations_unique:
        df_station = df.loc[df["station_id"] == station]
        max_stands = df.loc[df["station_id"] == station]["max_stands"].max()
        X = df_station.drop(['station_id', 'available_bikes', 'available_bike_stands', 'availabilty_ratio', 'max_stands', 'bikes_flag', 'stands_flag'],axis=1)
        y = df_station['availabilty_ratio']
        y_bike_chance = df_station['bikes_flag']
        y_stands_chance = df_station['stands_flag']
        
        regr = RandomForestRegressor(n_estimators = 100, random_state = 42)
        regr.fit(X, y)

        regr_bike_chance = RandomForestRegressor(n_estimators = 100, random_state = 42)
        regr_bike_chance.fit(X, y_bike_chance)

        regr_stands_chance = RandomForestRegressor(n_estimators = 100, random_state = 42)
        regr_stands_chance.fit(X, y_stands_chance)

        prediction_models[station] = [regr.predict, max_stands, regr_bike_chance.predict, regr_stands_chance.predict]

        with open(f'station_models/predictionModels_{station}.pkl', 'wb') as file:
            pickle.dump([regr.predict, max_stands, regr_bike_chance.predict, regr_stands_chance.predict], file)

: 

--------------------------------------